In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
from scipy import stats

In [2]:
import Sim_random_pick_out_of_battery_trip_dest_selection
from datetime import timedelta as td
from datetime import datetime as time

suppose we just need a subset of stations to be ebike stations

In [30]:
#p = [0.1,0.2,0.3,0.4,0.5]
#p = [0.6,0.7,0.8,0.9]
p = [0.8,0.9,0.95]

stations_ranked is a data file that sorted the stations by their demand (popularity) (= tripsin+tripsout+tripsfailedin+tripsfailedout)

In [31]:
stations_ranked = pd.read_csv("stations.csv")
stations_ranked.head()

,stationid,tripsin,tripsout,tripsfailedin,tripsfailedout,tripsfailedout_battery,trips_failedout_destinationfull,station_capacity,demand,percent
0,519,68062,68086,66,13003,0,1723,61,149217,0.008820
1,426,59633,59645,3078,8907,0,1336,29,131263,0.007759
2,497,54780,54794,578,5234,0,1195,59,115386,0.006820
3,514,48941,48929,1088,9459,0,855,52,108417,0.006408
4,435,49798,49817,160,5577,0,997,47,105352,0.006227


In [32]:
total_stations = len(stations_ranked)
total_stations

823

In [33]:
initial_stations=eval(open(("stations_initial.txt")).read())

Still assume 40% of the fleet size have e-bikes. Also we assume that if a station is chosen to be ebike station, then all docks are electrified.

The code in the following block is trying to get the steady state after 30-week simulation, assuming we choose p percent of stations to be electrified. The first p-percent of stations in the stations_ranked list was selected to be the ebike stations in this simulation.

In [34]:
from copy import deepcopy
initial_stations_copy = deepcopy(initial_stations)

In [35]:
num_ebike = 0.4
for percent in p:
    # number of stations we pick to electrify
    num_estation = int(total_stations*percent)
    subset = stations_ranked['stationid'][0:num_estation]
    
    for station in initial_stations.keys():
        initial_stations[station]['ecap']= initial_stations[station]['cap']
        initial_stations[station]['ebike']=round(initial_stations[station]['ecap']*num_ebike)
        initial_stations[station]['bike']=0
        initial_stations[station]['cap']=0
        if station in subset:
            initial_stations[station]['edock'] = initial_stations[station]['ecap']
        else:
            initial_stations[station]['edock'] = 0
    
    start_time = time(2017,7,1,hour= 6)
    end_time=start_time+ td(weeks=30)
    gc=Sim_random_pick_out_of_battery_trip_dest_selection.GlobalClock(start_time,end_time,initial_stations)
    gc.clockAdvance() 
    steady = {}
    for i in gc.stations.keys(): 
        temp={}   
        temp['ebike'] = len(gc.stations[i].ebike)
        temp['ecap'] = gc.stations[i].ebike_cap
        temp['edock'] = gc.stations[i].edock
        temp['bike'] = 0
        temp['cap'] = 0
        steady[i] = temp

    f = open("steady_states_p/stations_initial_"+str(percent)+".txt","w")
    f.write(str(steady))
    f.close()
    initial_stations = deepcopy(initial_stations_copy)

the total number of ebikes: 10149
the total number of regular bikes: 0
2017-07-01 06:00:02.217335 0
2017-07-08 06:00:00.569163 1
2017-07-15 06:00:04.231489 2
2017-07-22 06:00:00.279842 3
2017-07-29 06:00:01.907117 4
2017-08-05 06:00:04.153920 5
2017-08-12 06:00:04.983133 6
2017-08-19 06:00:03.492337 7
2017-08-26 06:00:09.362010 8
2017-09-02 06:00:00.235340 9
2017-09-09 06:00:00.447500 10
2017-09-16 06:00:03.232987 11
2017-09-23 06:00:01.510488 12
2017-09-30 06:00:01.292404 13
2017-10-07 06:00:01.669361 14
2017-10-14 06:00:01.678177 15
2017-10-21 06:00:01.822705 16
2017-10-28 06:00:00.703844 17
2017-11-04 06:00:00.292488 18
2017-11-11 06:00:03.018000 19
2017-11-18 06:00:00.572569 20
2017-11-25 06:00:00.005591 21
2017-12-02 06:00:00.441256 22
2017-12-09 06:00:00.087041 23
2017-12-16 06:00:02.413116 24
2017-12-23 06:00:00.461979 25
2017-12-30 06:00:01.275605 26
2018-01-06 06:00:02.243513 27
2018-01-13 06:00:00.752414 28
2018-01-20 06:00:02.692846 29
2018-01-27 06:00:05.993212 30
the total